In [17]:
import pytorch_lightning as pl
import torch as th
import ChemtabUQ

In [18]:
ckpt_path='../CT_logs_Mu/MAPE-PCA-CT/version_13420071/checkpoints/epoch=10891-step=10892.ckpt'
PL_module = ChemtabUQ.FFRegressor.load_from_checkpoint(ckpt_path, input_size=25)

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [ ]:
import os
from glob import glob
print(os.getcwd())
print(glob('../CT_logs_Mu/MAPE-PCA-CT/*/checkpoints/*'))

In [ ]:
onnx_model_path = os.path.dirname(ckpt_path)+'/model.onnx'
PL_module.to_onnx(onnx_model_path)

In [ ]:
import onnx
from onnx_tf.backend import prepare
onnx_model = onnx.load(onnx_model_path)
tf_rep = prepare(onnx_model)
tf_rep.export_graph(os.path.dirname(ckpt_path)+'/model.pb')

In [ ]:
"""%pip install onnx
%pip install onnx_tf
%pip install tensorflow_probability"""

%pip uninstall -y tensorflow
%pip install tensorflow
%pip uninstall -y tensorflow_probability
%pip install tensorflow_probability==0.19.0

In [ ]:
%pip show tensorflow_probability

In [3]:
%pip install pytorch-lightning

Note: you may need to restart the kernel to use updated packages.


In [15]:
!pip install pandas scikit-learn

  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/63/87/6cd5450f0385966bf2a5b865a2043cf68c2a41676193afdbccb40f8719dc/scikit_learn-1.3.0-cp39-cp39-macosx_12_0_arm64.whl.metadata
  Obtaining dependency information for scipy>=1.5.0 from https://files.pythonhosted.org/packages/77/31/b063f21370c6050a663aae5a9868d2fe112b21caeface3c248016dfea092/scipy-1.11.2-cp39-cp39-macosx_12_0_arm64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 kB 2.5 MB/s eta 0:00:00
  Obtaining dependency information for joblib>=1.1.1 from https://files.pythonhosted.org/packages/10/40/d551139c85db202f1f384ba8bcf96aca2f329440a844f924c8a0040b6d02/joblib-1.3.2-py3-none-any.whl.metadata
  Obtaining dependency information for threadpoolctl>=2.0.0 from https://files.pythonhosted.org/packages/81/12/fd4dea011af9d69e1cad05c75f3f7202cdcbeac9b712eea58ca779a72865/threadpoolctl-3.2.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 5

In [16]:
import sklearn